In [3]:
import findspark

findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

from pyspark.sql.functions import *

### Joins

In [18]:
empleados = spark.read.parquet('./data/joins/empleados.parquet')
departamentos = spark.read.parquet('./data/joins/departamentos.parquet')

In [20]:
empleados.show()

+------+--------+
|nombre|num_dpto|
+------+--------+
|  Luis|      33|
| Katia|      33|
|  Raul|      34|
| Pedro|       0|
| Laura|      34|
|Sandro|      31|
+------+--------+



In [21]:
departamentos.show()

+---+-----------+
| id|nombre_dpto|
+---+-----------+
| 31|     letras|
| 33|    derecho|
| 34| matemática|
| 35|informática|
+---+-----------+



In [26]:
join_df = empleados.join(departamentos, col('num_dpto') == col('id')).show()

+------+--------+---+-----------+
|nombre|num_dpto| id|nombre_dpto|
+------+--------+---+-----------+
|  Luis|      33| 33|    derecho|
| Katia|      33| 33|    derecho|
|  Raul|      34| 34| matemática|
| Laura|      34| 34| matemática|
|Sandro|      31| 31|     letras|
+------+--------+---+-----------+



In [25]:
join_df = empleados.join(departamentos).where(col('num_dpto') == col('id')).show()

+------+--------+---+-----------+
|nombre|num_dpto| id|nombre_dpto|
+------+--------+---+-----------+
|  Luis|      33| 33|    derecho|
| Katia|      33| 33|    derecho|
|  Raul|      34| 34| matemática|
| Laura|      34| 34| matemática|
|Sandro|      31| 31|     letras|
+------+--------+---+-----------+



### Left Outer Join

In [29]:
empleados.join(departamentos, col('num_dpto') == col('id'), 'leftouter').show()     #se puede usar 'leftouter', 'left_outer' o 'left'

+------+--------+----+-----------+
|nombre|num_dpto|  id|nombre_dpto|
+------+--------+----+-----------+
|  Luis|      33|  33|    derecho|
| Katia|      33|  33|    derecho|
|  Raul|      34|  34| matemática|
| Pedro|       0|null|       null|
| Laura|      34|  34| matemática|
|Sandro|      31|  31|     letras|
+------+--------+----+-----------+



### Left Outer Join

In [30]:
empleados.join(departamentos, col('num_dpto') == col('id'), 'rightouter').show()    #se puede usar 'rightouter', 'right_outer' o 'right'

+------+--------+---+-----------+
|nombre|num_dpto| id|nombre_dpto|
+------+--------+---+-----------+
|Sandro|      31| 31|     letras|
| Katia|      33| 33|    derecho|
|  Luis|      33| 33|    derecho|
| Laura|      34| 34| matemática|
|  Raul|      34| 34| matemática|
|  null|    null| 35|informática|
+------+--------+---+-----------+



### Full Outer Join

In [40]:
empleados.join(departamentos, col('num_dpto') == col('id'), 'outer').show()     # se puede usar 'outer', 'full', 'fullouter' o 'full_outer'

+------+--------+----+-----------+
|nombre|num_dpto|  id|nombre_dpto|
+------+--------+----+-----------+
| Pedro|       0|null|       null|
|Sandro|      31|  31|     letras|
|  Luis|      33|  33|    derecho|
| Katia|      33|  33|    derecho|
|  Raul|      34|  34| matemática|
| Laura|      34|  34| matemática|
|  null|    null|  35|informática|
+------+--------+----+-----------+



### Left Anti Join

In [32]:
empleados.join(departamentos, col('num_dpto') == col('id'), 'leftanti').show()  #se puede usar 'leftanti' o 'left_anti'

+------+--------+
|nombre|num_dpto|
+------+--------+
| Pedro|       0|
+------+--------+



### Left Semi Join

In [42]:
empleados.join(departamentos, col('num_dpto') == col('id'), 'leftsemi').show()  #se puede usar 'leftsemi' , 'semi' o 'left_semi'

+------+--------+
|nombre|num_dpto|
+------+--------+
|  Luis|      33|
| Katia|      33|
|  Raul|      34|
| Laura|      34|
|Sandro|      31|
+------+--------+



### Cross Join

In [47]:
empleados.crossJoin(departamentos).show()
empleados.crossJoin(departamentos).count()

+------+--------+---+-----------+
|nombre|num_dpto| id|nombre_dpto|
+------+--------+---+-----------+
|  Luis|      33| 31|     letras|
|  Luis|      33| 33|    derecho|
|  Luis|      33| 34| matemática|
|  Luis|      33| 35|informática|
| Katia|      33| 31|     letras|
| Katia|      33| 33|    derecho|
| Katia|      33| 34| matemática|
| Katia|      33| 35|informática|
|  Raul|      34| 31|     letras|
|  Raul|      34| 33|    derecho|
|  Raul|      34| 34| matemática|
|  Raul|      34| 35|informática|
| Pedro|       0| 31|     letras|
| Pedro|       0| 33|    derecho|
| Pedro|       0| 34| matemática|
| Pedro|       0| 35|informática|
| Laura|      34| 31|     letras|
| Laura|      34| 33|    derecho|
| Laura|      34| 34| matemática|
| Laura|      34| 35|informática|
+------+--------+---+-----------+
only showing top 20 rows



24

### Manejo de nombres de columna duplicados

In [48]:
deps = departamentos.withColumn('num_dpto', col('id'))

In [49]:
deps.printSchema()

empleados.printSchema()

root
 |-- id: long (nullable = true)
 |-- nombre_dpto: string (nullable = true)
 |-- num_dpto: long (nullable = true)

root
 |-- nombre: string (nullable = true)
 |-- num_dpto: long (nullable = true)



In [50]:
empleados.join(deps, empleados['num_dpto'] == deps['num_dpto']).show()

+------+--------+---+-----------+--------+
|nombre|num_dpto| id|nombre_dpto|num_dpto|
+------+--------+---+-----------+--------+
|Sandro|      31| 31|     letras|      31|
| Katia|      33| 33|    derecho|      33|
|  Luis|      33| 33|    derecho|      33|
| Laura|      34| 34| matemática|      34|
|  Raul|      34| 34| matemática|      34|
+------+--------+---+-----------+--------+



In [51]:
empleados.join(deps, 'num_dpto').show()

+--------+------+---+-----------+
|num_dpto|nombre| id|nombre_dpto|
+--------+------+---+-----------+
|      31|Sandro| 31|     letras|
|      33| Katia| 33|    derecho|
|      33|  Luis| 33|    derecho|
|      34| Laura| 34| matemática|
|      34|  Raul| 34| matemática|
+--------+------+---+-----------+

